In [1]:
import numpy as np
from IPython.display import display, Image
%matplotlib inline
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
from astropy.io import fits
from astropy.table import Table
import pandas as pd
from hmmlearn import hmm
import numpy as np
import seaborn as sns
from scipy.stats import norm
import itertools
import matplotlib.pyplot as plt
from hmmlearn.hmm import GMMHMM
from  sklearn.metrics import mean_squared_error as mse
from astropy.io import fits
import matplotlib.pyplot as plt
import os, sys
dirpath = os.getcwd().split("code")[0]+"code/"
sys.path.append(dirpath)
from pre_process import clean_LC

def plot_state_matrix(cm, states,title='Transition Probabilities',cmap=plt.cm.Blues):
    plt.figure(figsize=(10,10))
    plt.imshow(cm, interpolation='nearest', cmap=cmap)
    plt.title(title)
    plt.colorbar()
    tick_marks = np.arange(len(states))
    plt.xticks(tick_marks, states)
    plt.yticks(tick_marks, states)
    for i, j in itertools.product(range(cm.shape[0]), range(cm.shape[1])):
        plt.text(j, i, format(cm[i, j], '.3f'),
                 horizontalalignment="center",
                 color="white" if cm[i, j] > cm.max() / 2. else "black")
    plt.ylabel('Origin state')
    plt.xlabel('Destination state')
    plt.tight_layout()

    
def plot_model_and_data(plot_lc, model, p_n=2000, real_lc=[] ):
    X_state = model.predict(plot_lc) #predice el estado, no el valor 
    X_state_mean = model.means_[X_state][:,0]
    X_state_cov = np.sqrt(model.covars_[X_state][:,0,0])
    if len(real_lc) != 0:
        if len(plot_lc) != len(real_lc):
            mask_nan = np.isnan(real_lc)
            aux = np.tile(np.nan, len(real_lc) )
            
            aux_lc = aux.copy()
            aux_lc[~mask_nan] = np.squeeze(plot_lc)
            plot_lc = aux_lc
            
            aux_X_state_mean = aux.copy()
            aux_X_state_mean[~mask_nan] = X_state_mean
            X_state_mean = aux_X_state_mean
            
            aux_X_state_cov = aux.copy()
            aux_X_state_cov[~mask_nan] = X_state_cov
            X_state_cov = aux_X_state_cov
    plt.figure(figsize=(14,6))
    plt.plot(plot_lc,'r.-',label="Data")
    plt.fill_between(np.arange(len(plot_lc)),
                     X_state_mean-1.96*X_state_cov,
                     X_state_mean+1.96*X_state_cov) 
    plt.plot(X_state_mean,'g*-',label="Prediccion de la media") 
    plt.legend()
    plt.title("prediccion de markov en curva")
    plt.show()
    
    plt.figure(figsize=(14,6))
    plt.plot(plot_lc[:p_n],'r.-',label="Data")
    plt.fill_between(np.arange(p_n),
                     X_state_mean[:p_n]-1.96*X_state_cov[:p_n],
                     X_state_mean[:p_n]+1.96*X_state_cov[:p_n]) 
    plt.plot(X_state_mean[:p_n],'g*-',label="Prediccion de la media") 
    plt.legend()
    plt.title("prediccion de markov zoom en curva")
    plt.show()
    
    
#older = "/work/work_teamEXOPLANET/KOI_LC/"
df_meta=pd.read_csv('../../KOI_Data/kepler_dataset.csv')
df_meta.head()

,Kepler ID,KOI Name,Kepler Name,KOI Number,Kepler Disposition,NExScI Disposition,RA (J2000),RA Error,Dec (J2000),Dec Error,...,Contamination Flag,Insolation Flux,Insolation Flux err1,Insolation Flux err2,Fitted Stellar Density,Fitted Stellar Density err1,Fitted Stellar Density err2,Planetary Fit,Disposition Score,File Name
0,757450,K00889.01,Kepler-75 b,889.01,CANDIDATE,CONFIRMED,19 24 33.024,0.0,+36 34 38.57,0.0,...,0,83.21,16.69,-13.87,11.62061,0.20683,-0.40258,LS+MCMC,0.999,kplr000757450-20160128150956_dvt.fits
1,892772,K01009.01,NaN,1009.01,FALSE POSITIVE,FALSE POSITIVE,19 24 36.811,0.0,+36 40 43.83,0.0,...,0,119.70,48.69,-36.91,0.88510,0.10080,-0.39457,LS+MCMC,0.000,kplr000892772-20160128150956_dvt.fits
2,1025986,K07621.01,NaN,7621.01,FALSE POSITIVE,FALSE POSITIVE,19 24 08.086,0.0,+36 46 15.75,0.0,...,0,2.00,0.92,-0.76,33.55994,16.58298,-31.53180,LS+MCMC,0.000,kplr001025986-20160128150956_dvt.fits
3,1026032,K06252.01,NaN,6252.01,FALSE POSITIVE,FALSE POSITIVE,19 24 10.577,0.0,+36 43 45.38,0.0,...,0,98.60,52.46,-21.18,0.73874,0.00021,-0.00735,LS+MCMC,0.000,kplr001026032-20160128150956_dvt.fits
4,1026957,K00958.01,NaN,958.01,CANDIDATE,CANDIDATE,19 25 01.078,0.0,+36 44 37.00,0.0,...,0,13.18,3.00,-1.98,102.86534,5.47239,-30.25813,LS+MCMC,1.000,kplr001026957-20160128150956_dvt.fits


In [2]:
! ls '/work/work_teamEXOPLANET/KOI_LC/cleaned/'

LC_kepler_processed.npy  LC_kepler_processed_old.npy


In [3]:
falsos_pos=df_meta.loc[df_meta['NExScI Disposition']=='FALSE POSITIVE']
falsos_pos

,Kepler ID,KOI Name,Kepler Name,KOI Number,Kepler Disposition,NExScI Disposition,RA (J2000),RA Error,Dec (J2000),Dec Error,...,Contamination Flag,Insolation Flux,Insolation Flux err1,Insolation Flux err2,Fitted Stellar Density,Fitted Stellar Density err1,Fitted Stellar Density err2,Planetary Fit,Disposition Score,File Name
1,892772,K01009.01,NaN,1009.01,FALSE POSITIVE,FALSE POSITIVE,19 24 36.811,0.0,+36 40 43.83,0.0,...,0,119.70,48.69,-36.91,0.88510,0.10080,-0.39457,LS+MCMC,0.000,kplr000892772-20160128150956_dvt.fits
2,1025986,K07621.01,NaN,7621.01,FALSE POSITIVE,FALSE POSITIVE,19 24 08.086,0.0,+36 46 15.75,0.0,...,0,2.00,0.92,-0.76,33.55994,16.58298,-31.53180,LS+MCMC,0.000,kplr001025986-20160128150956_dvt.fits
3,1026032,K06252.01,NaN,6252.01,FALSE POSITIVE,FALSE POSITIVE,19 24 10.577,0.0,+36 43 45.38,0.0,...,0,98.60,52.46,-21.18,0.73874,0.00021,-0.00735,LS+MCMC,0.000,kplr001026032-20160128150956_dvt.fits
6,1293031,K04162.01,NaN,4162.01,FALSE POSITIVE,FALSE POSITIVE,19 24 05.033,0.0,+36 54 13.13,0.0,...,0,149399.93,35721.31,-106451.43,0.06737,0.00658,-0.00461,LS+MCMC,0.000,kplr001293031-20160128150956_dvt.fits
7,1293046,K07622.01,NaN,7622.01,FALSE POSITIVE,FALSE POSITIVE,19 24 06.372,0.0,+36 54 58.93,0.0,...,0,3.44,0.94,-0.97,34.69388,13.11193,-21.13275,LS+MCMC,0.000,kplr001293046-20160128150956_dvt.fits
9,1432214,K00998.01,NaN,998.01,FALSE POSITIVE,FALSE POSITIVE,19 25 31.846,0.0,+37 04 22.76,0.0,...,0,4.68,2.07,-1.12,17.58377,0.76473,-0.48796,LS+MCMC,0.000,kplr001432214-20160128150956_dvt.fits
12,1433962,K00982.01,NaN,982.01,FALSE POSITIVE,FALSE POSITIVE,19 27 02.870,0.0,+37 05 55.10,0.0,...,1,290.51,87.21,-73.46,0.09265,0.00000,0.00000,LS,0.000,kplr001433962-20160128150956_dvt.fits
13,1433980,K03924.01,NaN,3924.01,FALSE POSITIVE,FALSE POSITIVE,19 27 03.919,0.0,+37 05 49.89,0.0,...,0,6128.09,4886.70,-3736.14,0.06389,0.00061,-0.00083,LS+MCMC,0.000,kplr001433980-20160128150956_dvt.fits
14,1434395,K07623.01,NaN,7623.01,FALSE POSITIVE,FALSE POSITIVE,19 27 25.555,0.0,+37 03 56.34,0.0,...,0,93172.12,12806.48,-67878.27,1.00737,0.29745,-0.56373,LS+MCMC,0.014,kplr001434395-20160128150956_dvt.fits
15,1571511,K00362.01,NaN,362.01,FALSE POSITIVE,FALSE POSITIVE,19 23 59.261,0.0,+37 11 57.19,0.0,...,0,101.92,91.75,-33.91,0.31052,0.00269,-0.00269,LS+MCMC,0.004,kplr001571511-20160128150956_dvt.fits


In [4]:
index_fp=list(falsos_pos.index)
len(index_fp)

3976

In [5]:
kplr_ids=np.asarray(df_meta['Kepler ID'].values)[[index_fp]]
kplr_ids.shape

/users/mbugueno/.local/lib/python3.6/site-packages/ipykernel_launcher.py:1: FutureWarning: Using a non-tuple sequence for multidimensional indexing is deprecated; use `arr[tuple(seq)]` instead of `arr[seq]`. In the future this will be interpreted as an array index, `arr[np.array(seq)]`, which will result either in an error or a different result.
  """Entry point for launching an IPython kernel.


(3976,)

In [6]:
kois_name=np.asarray(df_meta['KOI Name'].values)[[index_fp]]
kois_name.shape

/users/mbugueno/.local/lib/python3.6/site-packages/ipykernel_launcher.py:1: FutureWarning: Using a non-tuple sequence for multidimensional indexing is deprecated; use `arr[tuple(seq)]` instead of `arr[seq]`. In the future this will be interpreted as an array index, `arr[np.array(seq)]`, which will result either in an error or a different result.
  """Entry point for launching an IPython kernel.


(3976,)

### recuperando FP 20 estados

In [7]:
our_detrends=np.load('/work/work_teamEXOPLANET/KOI_LC/cleaned/LC_kepler_processed.npy')
print (our_detrends.shape)
our_detrends=our_detrends[index_fp]
our_detrends.shape

(8054, 71427)


(3976, 71427)

In [8]:
folder_marg='/work/work_teamEXOPLANET/MTF/'

In [ ]:
import os

i=0
for lc_our_detrend in our_detrends:  
    recuperado=False
    name_kepler=kois_name[i]
    id_kepler=kplr_ids[i]    
    try:
        a=np.load(folder_marg+'falsos_positivos_20/'+str(name_kepler)+'.npy')
        print ("matriz ", name_kepler, 'recuperada previamente')
        i+=1
        continue
    
    except:         
        print ("Calculando matriz para ", name_kepler, "pues tenia kepler id",id_kepler, "con i:",i)
        lc_wind_nan = []
        lengths = []
        sublist = []
        for value in lc_our_detrend:
            if np.isnan(value) and len(sublist) != 0:
                lc_wind_nan.append(np.asarray(sublist))
                lengths.append(len(sublist))
                sublist = []
            elif np.isnan(value) and len(sublist) == 0:
                continue
            else: 
                sublist.append(value) 
        if len(sublist) != 0:      
            lc_wind_nan.append(np.asarray(sublist))
            lengths.append(len(sublist))
        lc_wind_nan = np.asarray(lc_wind_nan)   
        lengths = np.asarray(lengths)   

        lc_wind_nan = np.concatenate(lc_wind_nan).reshape(-1,1)
        lc_wind_nan = lc_wind_nan/np.abs(np.min(lc_wind_nan)) 
        n_sta=20
        markov_model = hmm.GaussianHMM(n_components=n_sta, n_iter=50)
        markov_model.fit(lc_wind_nan , lengths)
        np.save(folder_marg+'falsos_positivos_20/'+str(name_kepler)+'.npy', markov_model.transmat_)
        i+=1


Calculando matriz para  K01009.01 pues tenia kepler id 892772 con i: 0
Calculando matriz para  K07621.01 pues tenia kepler id 1025986 con i: 1
Calculando matriz para  K06252.01 pues tenia kepler id 1026032 con i: 2
Calculando matriz para  K04162.01 pues tenia kepler id 1293031 con i: 3
Calculando matriz para  K07622.01 pues tenia kepler id 1293046 con i: 4
Calculando matriz para  K00998.01 pues tenia kepler id 1432214 con i: 5
Calculando matriz para  K00982.01 pues tenia kepler id 1433962 con i: 6
Calculando matriz para  K03924.01 pues tenia kepler id 1433980 con i: 7
Calculando matriz para  K07623.01 pues tenia kepler id 1434395 con i: 8
Calculando matriz para  K00362.01 pues tenia kepler id 1571511 con i: 9
Calculando matriz para  K04924.01 pues tenia kepler id 1572948 con i: 10
Calculando matriz para  K03658.01 pues tenia kepler id 1575690 con i: 11
Calculando matriz para  K00993.03 pues tenia kepler id 1718189 con i: 12
Calculando matriz para  K01008.01 pues tenia kepler id 1722276

Calculando matriz para  K06285.01 pues tenia kepler id 2693092 con i: 113
Calculando matriz para  K04600.01 pues tenia kepler id 2694632 con i: 114
Calculando matriz para  K00400.01 pues tenia kepler id 2695110 con i: 115
Calculando matriz para  K03161.01 pues tenia kepler id 2696703 con i: 116
Calculando matriz para  K03853.01 pues tenia kepler id 2697935 con i: 117
Calculando matriz para  K06095.01 pues tenia kepler id 2707097 con i: 118
Calculando matriz para  K04950.01 pues tenia kepler id 2707916 con i: 119
Calculando matriz para  K03253.01 pues tenia kepler id 2707985 con i: 120
Calculando matriz para  K06286.01 pues tenia kepler id 2708156 con i: 121
Calculando matriz para  K06287.01 pues tenia kepler id 2708172 con i: 122
Calculando matriz para  K04045.01 pues tenia kepler id 2708203 con i: 123
Calculando matriz para  K04352.01 pues tenia kepler id 2708221 con i: 124
Calculando matriz para  K04951.01 pues tenia kepler id 2708270 con i: 125
Calculando matriz para  K04102.01 pues

Calculando matriz para  K00966.01 pues tenia kepler id 3233043 con i: 224
Calculando matriz para  K07649.01 pues tenia kepler id 3235282 con i: 225
Calculando matriz para  K01093.01 pues tenia kepler id 3239636 con i: 226
Calculando matriz para  K04968.01 pues tenia kepler id 3240049 con i: 227
Calculando matriz para  K03515.01 pues tenia kepler id 3240159 con i: 228
Calculando matriz para  K03185.01 pues tenia kepler id 3240406 con i: 229
Calculando matriz para  K01098.01 pues tenia kepler id 3240706 con i: 230
Calculando matriz para  K04141.01 pues tenia kepler id 3241248 con i: 231
Calculando matriz para  K05984.01 pues tenia kepler id 3241344 con i: 232
Calculando matriz para  K02863.01 pues tenia kepler id 3241557 con i: 233
Calculando matriz para  K02824.01 pues tenia kepler id 3241604 con i: 234
Calculando matriz para  K06312.01 pues tenia kepler id 3241619 con i: 235
Calculando matriz para  K07650.01 pues tenia kepler id 3241647 con i: 236
Calculando matriz para  K06313.01 pues